In [1]:
import tensorflow as tf
import pandas as pd
import shutil

print(tf.__version__)
tf.enable_eager_execution()

1.15.0


In [2]:
import math

def volume(r: float, h: float) -> float:
    return math.pi * r ** 2 * h

volume(2, 3)

37.69911184307752

In [3]:
import numpy as np

def rand(data_size, min_val, max_val):
    r = np.random.random(data_size)
    return list((r + min_val) * (max_val - min_val))

rand(3, 0.5, 2.0)

[1.8982271001525828, 1.1219003001790622, 0.7562935239010877]

In [4]:
data_size = 10000

inputs = {
    'r': rand(data_size, 0.5, 2.0),
    'h': rand(data_size, 0.5, 2.0),
}
outputs = np.array([volume(inputs['r'][i], inputs['h'][i]) for i in range(len(inputs['r']))])
#print(inputs)
#print(outputs)

In [12]:
eval_data_size = 1000
eval_inputs = {
    'r': rand(eval_data_size, 0.5, 2.0),
    'h': rand(eval_data_size, 0.5, 2.0),
}
eval_outputs = np.array([volume(eval_inputs['r'][i], eval_inputs['h'][i]) for i in range(len(eval_inputs['r']))])


In [6]:
CY_OUT_DIR = './cy_model'

shutil.rmtree(path = CY_OUT_DIR, ignore_errors = True)

cy_feature_columns = [
    tf.feature_column.numeric_column('r'),
    tf.feature_column.numeric_column('h'),
]
model = tf.estimator.DNNRegressor(
    feature_columns=cy_feature_columns,
    hidden_units=[10, 10],
    model_dir=CY_OUT_DIR,
    config=tf.estimator.RunConfig(tf_random_seed = 1)
)

INFO:tensorflow:Using config: {'_service': None, '_train_distribute': None, '_save_checkpoints_steps': None, '_eval_distribute': None, '_tf_random_seed': 1, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_task_id': 0, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd16a34e780>, '_is_chief': True, '_global_id_in_cluster': 0, '_protocol': None, '_save_summary_steps': 100, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_device_fn': None, '_evaluation_master': '', '_num_worker_replicas': 1, '_session_creation_timeout_secs': 7200, '_model_dir': './cy_model', '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600}


In [7]:
def cy_input_fn():
    return tf.data.Dataset.from_tensor_slices((
        inputs,
        outputs,
    )).repeat().shuffle(1024).batch(128)
    # .shuffle(1024).repeat().batch(128)

def cy_eval_input_fn():
    return tf.data.Dataset.from_tensor_slices((
        eval_inputs,
        eval_outputs,
    )).batch(128)

In [14]:
model.train(cy_input_fn, steps=1000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./cy_model/model.ckpt.
INFO:tensorflow:loss = 21484.877, step = 0
INFO:tensorflow:global_step/sec: 287.199
INFO:tensorflow:loss = 2888.6177, step = 100 (0.353 sec)
INFO:tensorflow:global_step/sec: 342.503
INFO:tensorflow:lo

In [15]:
metrics = model.evaluate(cy_eval_input_fn)
metrics['average_loss']**0.5

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-21T07:12:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cy_model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-21-07:12:37
INFO:tensorflow:Saving dict for global step 1000: average_loss = 1.99253, global_step = 1000, label/mean = 11.559625, loss = 249.06625, prediction/mean = 11.34709
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ./cy_model/model.ckpt-1000


1.4115700437062553